# Exploring and Analyzing MovieLens Dataset
Here the MovieLens dataset available on https://grouplens.org/datasets/movielens/ was used and performed some Alaysis using Python, Pandas, and Matplotlib. 

## Data Loading and Overview
• Load the movies.csv and ratings.csv files into separate Pandas DataFrames. Display the 
first 5 rows of each DataFrame. <br>
• Check for missing data in both DataFrames. Provide a summary showing the total number 
of missing values for each column.

In [1]:
import pandas as pd
#import matplotlib.pyplot  as plt
mov_data = pd.read_csv("movies.csv")
rating_data = pd.read_csv("ratings.csv")
tags_data = pd.read_csv('tags.csv')

In [2]:
mov_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
rating_data.head(5)

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


Below is the summary showing the total number of missing values for each column.

In [4]:
movies_missing = mov_data.isnull().sum()
movies_missing

movieId    0
title      0
genres     0
dtype: int64

In [5]:
rating_missing = rating_data.isnull().sum()
rating_missing

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

## Data Cleaning and Preparation
• Extract the release year from the title column in movies.csv and create a new column called 
year. <br>
• Example: For "Toy Story (1995)", extract 1995. <br>
• Replace all missing years in the dataset with Unknown. <br>
• Identify and remove duplicate entries, if any, from both datasets. <br>
• In the ratings.csv file, check for users who have rated less than 5 movies. Remove these 
users from the dataset.

In [6]:
mov_data['year'] = mov_data['title'].str.extract(r"\((\d{4})\)")
mov_data

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama,2022
87581,292737,Shelter in Solitude (2023),Comedy|Drama,2023
87582,292753,Orca (2023),Drama,2023
87583,292755,The Angry Breed (1968),Drama,1968


In [ ]:
mov_data.fillna("Unknown", inplace=True)
mov_data

In [ ]:
mov_data.drop_duplicates(inplace=True)
rating_data.drop_duplicates(inplace=True)

In [ ]:
user_rating_counts = rating_data['userId'].value_counts()
user_rating_counts

In [ ]:
users_to_keep = user_rating_counts[user_rating_counts >= 5].index
print("Indexes are:\n",users_to_keep)
filtered_ratings_data = rating_data[rating_data['userId'].isin(users_to_keep)]
filtered_ratings_data

## Data Exploration and Transformation 
• Group the ratings.csv dataset by movieId and calculate the average rating for each movie. <br>
Add this information to the movies.csv dataset as a new column called average_rating.<br>
• Find the top 10 highest-rated movies (with at least 50 ratings). Display their titles, genres, 
and average ratings.<br>
• Identify the 5 most common genres in the dataset.<br>
• Hint: Use the genres column and split the pipe-separated values

In [7]:
ratings_summary = rating_data.groupby('movieId').agg(average_rating=('rating', 'mean'),rating_count=('rating', 'count')).reset_index()
ratings_summary

,movieId,average_rating,rating_count
0,1,3.897438,68997
1,2,3.275758,28904
2,3,3.139447,13134
3,4,2.845331,2806
4,5,3.059602,13154
...,...,...,...
84427,292731,4.000000,1
84428,292737,1.500000,1
84429,292753,4.000000,1
84430,292755,1.000000,1


In [ ]:
mov_data = mov_data.merge(ratings_summary, on='movieId')
mov_data

In [ ]:
top_movies = mov_data[mov_data['rating_count'] >= 50].sort_values(by='average_rating', ascending=False).head(10)
print("Top 10 Highest-Rated Movies (with at least 50 ratings):")
print(top_movies[['title', 'genres', 'average_rating']])

In [ ]:
all_genres = mov_data['genres'].str.split('|').explode()
genre_counts = all_genres.value_counts()
top_genres = genre_counts.head(5)
top_genres

## Data Aggregation and Group Operations 
• Calculate the total number of movies released each year. Create a bar chart to show the 
number of movies released by year. <br>
• Find the user who has rated the most movies and calculate their average rating.<br>
• Group movies by decade (e.g., 1990-1999, 2000-2009) and calculate:<br>
• The total number of movies released per decade.<br>
• The average rating of movies for each decade.<br>
• Visualize both metrics using separate bar charts

In [12]:
mov_data['year'] = mov_data['title'].str.extract(r'\((\d{4})\)').astype(float)
yearly_movie_count = mov_data.groupby('year')['movieId'].count()

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(yearly_movie_count.index, yearly_movie_count.values, color='skyblue')
plt.title('Number of Movies Released Each Year', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.show()

In [ ]:
user_rating_counts = rating_data.groupby('userId')['rating'].count()
most_active_user = user_rating_counts.idxmax()
most_active_user_avg_rating = rating_data[rating_data['userId'] == most_active_user]['rating'].mean()

print(f"User {most_active_user} rated the most movies ({user_rating_counts[most_active_user]} ratings) with an average rating of {most_active_user_avg_rating:.2f}.")

In [ ]:
mov_data['decade'] = (mov_data['year'] // 10 * 10).dropna()
decade_movie_counts = mov_data.groupby('decade')['movieId'].count()
decade_avg_ratings = rating_data.merge(mov_data, on='movieId').groupby('decade')['rating'].mean()

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(decade_movie_counts.index, decade_movie_counts.values, color='orange')
plt.title('Number of Movies Released Per Decade', fontsize=16)
plt.xlabel('Decade', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(decade_avg_ratings.index, decade_avg_ratings.values, color='green')
plt.title('Average Movie Rating Per Decade', fontsize=16)
plt.xlabel('Decade', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.show()

## Interactive Questions 
• Plot a histogram of average movie ratings, using bins of 0.5 (e.g., 0.0-0.5, 0.5-1.0, etc.). <br>
• Create a scatter plot of the average movie rating vs. the number of ratings each movie received. <br>
• Add appropriate labels and titles. <br>
• Identify any clear trends in the plot

In [ ]:
movie_avg_ratings = rating_data.groupby('movieId')['rating'].mean()
plt.figure(figsize=(12, 6))
plt.hist(movie_avg_ratings, bins=20, color='purple', edgecolor='black')
plt.title('Histogram of Average Movie Ratings', fontsize=16)
plt.xlabel('Average Rating', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

In [ ]:
movie_rating_counts = rating_data.groupby('movieId')['rating'].count()
plt.figure(figsize=(12, 6))
plt.scatter(movie_rating_counts, movie_avg_ratings, alpha=0.6, color='red')
plt.title('Average Movie Rating vs. Number of Ratings', fontsize=16)
plt.xlabel('Number of Ratings', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.show()

## Advanced Data Wrangling 
• Create a pivot table showing the average rating for each genre by year. Display only the top 5 genres (based on the count from Question 9). <br>
• Using the tags.csv file, find the most common tag applied to movies in the "Action" genre.

In [6]:
genres = mov_data['genres'].str.get_dummies(sep='|')
mov_data = pd.concat([mov_data, genres], axis=1)
top_5_genres = genres.sum().sort_values(ascending=False).head(5).index
ratings_movies = rating_data.merge(mov_data, on='movieId')
pivot_table = ratings_movies.pivot_table(index='year', values='rating', columns=top_5_genres, aggfunc='mean')

print("Pivot Table (Top 5 Genres by Year):")
print(pivot_table)

KeyError: 'year'

In [5]:
action_movies = mov_data[mov_data['genres'].str.contains('Action', na=False)]
common_tags = tags_data.merge(action_movies, on='movieId')
most_common_tag = common_tags['tag'].value_counts().idxmax()
print(f"The most common tag for 'Action' genre movies is: {most_common_tag}")

The most common tag for 'Action' genre movies is: action


## Correlation between the year and its average rating
• Create a heatmap to visualize the number of movies released by year and genre. <br>
• Hint: Use a pivot table and Seaborn’s heatmap. <br>
• Find out if there's any correlation between the year a movie was released and its average rating. <br>
• Use a scatter plot and compute the correlation coefficient

In [14]:
genre_movie_count = ratings_movies.pivot_table(index='year', columns='genres', values='movieId', aggfunc='count', fill_value=0)
plt.figure(figsize=(15, 10))
sns.heatmap(genre_movie_count, cmap='coolwarm', annot=False, fmt='d')
plt.title('Number of Movies Released by Year and Genre', fontsize=16)
plt.xlabel('Genre', fontsize=12)
plt.ylabel('Year', fontsize=12)
plt.show()

KeyError: 'year'

In [ ]:
year_avg_rating = ratings_movies.groupby('year')['rating'].mean()
plt.figure(figsize=(12, 6))
plt.scatter(year_avg_rating.index, year_avg_rating.values, color='magenta', alpha=0.7)
plt.title('Correlation Between Year and Average Movie Rating', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.show()

In [ ]:
year_correlation = year_avg_rating.corr(pd.Series(year_avg_rating.index))
print(f"Correlation coefficient between year and average rating: {year_correlation:.2f}")